In [17]:
import pandas as pd
import plotly.graph_objects as go
import utils

In [18]:
pair = "USD_JPY"
granularity = "H4"
df_raw = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [19]:
non_cols = ['time', 'volume']
mod_cols = [x for x in df_raw.columns if x not in non_cols]
df_raw[mod_cols] = df_raw[mod_cols].apply(pd.to_numeric)

In [20]:
df_raw.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2018-01-01T22:00:00.000000000Z,2968,112.666,112.793,112.582,112.723,112.626,112.786,112.559,112.716,112.706,112.800,112.601,112.730
1,2018-01-02T02:00:00.000000000Z,1009,112.720,112.751,112.666,112.688,112.712,112.744,112.657,112.683,112.728,112.758,112.674,112.694
2,2018-01-02T06:00:00.000000000Z,2226,112.691,112.700,112.249,112.252,112.684,112.694,112.242,112.245,112.698,112.705,112.256,112.258
3,2018-01-02T10:00:00.000000000Z,2056,112.254,112.282,112.086,112.124,112.247,112.275,112.079,112.119,112.260,112.288,112.092,112.130
4,2018-01-02T14:00:00.000000000Z,3781,112.122,112.304,112.058,112.285,112.114,112.297,112.051,112.278,112.129,112.311,112.065,112.292


In [21]:
SLOSS = 0.4
TPROFIT = 0.8
ENTRY_PRC = 0.1

def direction(row):
    if row.mid_c > row.mid_o:
        return 1
    return -1

def get_signal(row):
    if row.mid_h_prev > row.mid_h and row.mid_l_prev < row.mid_l:
        return row.DIRECTION_prev
    return 0

def get_entry_stop(row):
    if row.SIGNAL == 1:
        return (row.RANGE_prev * ENTRY_PRC) + row.mid_h_prev
    elif row.SIGNAL == -1:
        return row.mid_l_prev - (row.RANGE_prev * ENTRY_PRC)
    else:
        return 0

def get_stop_loss(row):
    if row.SIGNAL == 1:
        return row.mid_h_prev - (row.RANGE_prev * SLOSS)
    elif row.SIGNAL == -1:
        return row.mid_l_prev + (row.RANGE_prev * SLOSS)
    else:
        return 0


def get_take_profit(row):
    if row.SIGNAL == 1:
        return row.mid_h_prev + (row.RANGE_prev * TPROFIT)
    elif row.SIGNAL == -1:
        return row.mid_l_prev - (row.RANGE_prev * TPROFIT)
    else:
        return 0

In [22]:
df = df_raw[['time','mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_c', 'ask_c']].copy()
df['RANGE'] = df.mid_h - df.mid_l
df['mid_h_prev'] = df.mid_h.shift(1)
df['mid_l_prev'] = df.mid_l.shift(1)
df['RANGE_prev'] = df.RANGE.shift(1)
df['DIRECTION'] = df.apply(direction, axis=1)
df['DIRECTION_prev'] = df.DIRECTION.shift(1).fillna(0).astype(int)
df.dropna(inplace=True)
df['SIGNAL'] = df.apply(get_signal, axis=1)
df.reset_index(drop=True, inplace=True)

In [24]:
df['ENTRY'] = df.apply(get_entry_stop, axis=1)
df['STOPLOSS'] = df.apply(get_stop_loss, axis=1)
df['TAKEPROFIT'] = df.apply(get_take_profit, axis=1)

In [26]:
df[df.SIGNAL!=0].head()

,time,mid_o,mid_h,mid_l,mid_c,bid_c,ask_c,RANGE,mid_h_prev,mid_l_prev,RANGE_prev,DIRECTION,DIRECTION_prev,SIGNAL,ENTRY,STOPLOSS,TAKEPROFIT
0,2018-01-02T02:00:00.000000000Z,112.720,112.751,112.666,112.688,112.683,112.694,0.085,112.793,112.582,0.211,-1,1,1,112.8141,112.7086,112.9618
12,2018-01-04T02:00:00.000000000Z,112.731,112.731,112.618,112.661,112.654,112.668,0.113,112.778,112.480,0.298,-1,1,1,112.8078,112.6588,113.0164
17,2018-01-04T22:00:00.000000000Z,112.750,112.806,112.730,112.774,112.767,112.780,0.076,112.838,112.713,0.125,1,-1,-1,112.7005,112.7630,112.6130
21,2018-01-05T14:00:00.000000000Z,113.213,113.276,113.082,113.160,113.153,113.166,0.194,113.312,113.021,0.291,-1,-1,-1,112.9919,113.1374,112.7882
27,2018-01-08T14:00:00.000000000Z,112.928,113.125,112.904,113.082,113.076,113.088,0.221,113.167,112.884,0.283,1,-1,-1,112.8557,112.9972,112.6576


In [39]:
df_plot = df.iloc[200:300]
df_buys = df_plot[df_plot.SIGNAL == 1]
df_sells = df_plot[df_plot.SIGNAL == -1]

In [40]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
    line=dict(width=1), opacity=1,
    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))
fig.add_trace(go.Scatter(
    x=df_buys.time,
    y=df_buys.mid_c,
    mode='markers',
    marker=dict(color="#0ec4ce",size=12)
))
fig.add_trace(go.Scatter(
    x=df_sells.time,
    y=df_sells.mid_c,
    mode='markers',
    marker=dict(color="#043ef9",size=12)
))
fig.update_layout(width=1000,height=400,
    margin=dict(l=10,r=10,b=10,t=10),
    font=dict(size=10,color="#e1e1e1"),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
    gridcolor="#1f292f",
    showgrid=True,fixedrange=True,rangeslider=dict(visible=False),
    rangebreaks=[
        dict(bounds=["sat", "mon"])
    ]
)
fig.update_yaxes(
    gridcolor="#1f292f",
    showgrid=True
)
fig.show()